In [1]:
import sys
sys.path.append("../src/")

In [9]:
from code_parser import *
import pandas as pd
import tqdm
import networkx as nx
import numpy as np
import fasttext as ft
import pickle
from gensim.models import KeyedVectors as word2vec
from sklearn.model_selection import train_test_split

In [3]:
all_funcs_data_path = '../data/bcb_funcs_all.tsv'
parser_path = '../data/java.so'

In [4]:
all_funcs = pd.read_csv(all_funcs_data_path, delimiter="\t", header=None)
parser = get_parser(parser_path)

In [5]:
processed_functions = {}
for index, row in tqdm.notebook.tqdm(all_funcs.iterrows()):
    processed_function = parse_program(row[1], parser)
    processed_functions[row[0]] = processed_function

In [6]:
# with open("../data/all_funcs_parsed_ast.pkl", "wb") as f:
#     import pickle
#     pickle.dump(processed_functions, f)

In [7]:
# with open("../all_functions_parsed_ast.pkl", "rb") as f:
#     import pickle
#     processed_functions = pickle.load(f)

In [8]:
processed_functions2 = {} # nodes
# processed_functions3 = {} # numbered

for key in tqdm.notebook.tqdm(processed_functions):
    g = processed_functions[key]
    mapping = { node: node.name.lower() for node in g.nodes }
    
    g1 = nx.relabel_nodes(g, mapping)
    processed_functions2[key] = g1
    
    #g2 = nx.convert_node_labels_to_integers(g)
    #processed_functions3[key] = g2

In [22]:
with open("../data/all_funcs_parsed_ast_int_label_1.pkl", "wb") as f:
    import pickle
    pickle.dump(processed_functions3, f)

In [10]:
nodes = list()
for key in tqdm.notebook.tqdm(processed_functions2):
    nodes.extend(list(processed_functions2[key].nodes))

In [11]:
from collections import Counter, OrderedDict
freqs = OrderedDict({k: v for k, v in sorted(Counter(nodes).items(), reverse=True, key=lambda item: item[1])})

In [12]:
total_vocab_size = len(freqs)

In [13]:
import fasttext as ft
model = ft.load_model("fasttext.bin")

In [14]:
word2idx = {}
idx2word = {}
idx2vectors = {}
counter = 0
idx = 1
for key in freqs:
    token = key.lower().strip()
    word2idx[token] = idx
    idx2word[idx] = token
    idx2vectors[idx] = model[token]
    idx += 1

In [15]:
len(word2idx)

288180

In [16]:
processed_functions4 = {}
for key in tqdm.notebook.tqdm(processed_functions):
    g = processed_functions[key]
    
    for node in g.nodes:
        token = node.name.strip().lower()
        g.add_node(node, data=model[token], idx=word2idx[token])

    g1 = nx.convert_node_labels_to_integers(g)
    processed_functions4[key] = g1

In [28]:
for key in tqdm.notebook.tqdm(processed_functions4):
    g = processed_functions4[key]
    g = nx.to_undirected(g)
    nx.write_gpickle(g, f"../data/networkx_data_idx/{key}")

In [24]:
processed_functions5 = {}
for key in tqdm.notebook.tqdm(processed_functions):
    g = processed_functions[key]
    for node in g.nodes:
        token = node.name.strip().lower()
        g.add_node(node, idx=word2idx[token])
    g1 = nx.convert_node_labels_to_integers(g)
    processed_functions5[key] = g1

In [27]:
for key in tqdm.notebook.tqdm(processed_functions5):
    g = processed_functions5[key]
    g = nx.to_undirected(g)
    nx.write_gpickle(g, f"../data/networkx_idx/{key}")

In [25]:
! mkdir ../data/networkx_idx/

In [138]:
text = ""
for key in processed_functions:
    g = processed_functions[key]
    text += "\n" + " ".join([node.name.strip() for node in nx.dfs_preorder_nodes(g)])

In [139]:
f = open("../data/code.txt", "w")
f.write(text)
f.close()

In [1]:
import fasttext as ft
model = ft.train_unsupervised("../data/code.txt", dim=256, ws=10, minCount=3, wordNgrams=3, thread=40, epoch=10, verbose=5, lr=0.01)

In [5]:
model.get_nearest_neighbors("%quickref)

[(0.6981577277183533, 'addj'),
 (0.6771022081375122, 'addG'),
 (0.676001787185669, 'addNr'),
 (0.6736781001091003, 'addee'),
 (0.6717252731323242, 'addy'),
 (0.6594294905662537, 'addBLM'),
 (0.65911865234375, 'addIp'),
 (0.6573836207389832, 'addOn'),
 (0.6565410494804382, 'addKV'),
 (0.6522443890571594, 'addBPMs')]

In [34]:
model.save_model("../data/fasttext.bin")